In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Input, ConvLSTM3D, Conv3D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling3D, Concatenate, TimeDistributed, MaxPooling3D, AveragePooling3D, GlobalMaxPooling3D, LSTM, Lambda
from tensorflow.keras.models import Model
import numpy as np
import pickle as pkl

In [6]:
devices = tf.config.list_physical_devices()
print(devices)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [7]:
tf.config.set_visible_devices([], 'GPU')

In [4]:
# importing data
def returning_pkl_file_data(path : str):
    with open(path, 'rb') as f:
        temp = pkl.load(f)
    return temp

mci_func = returning_pkl_file_data(r"feature_extraction/MCI_func_52_79_95_79_197.pkl")
mci_struct = returning_pkl_file_data(r'feature_extraction/MCI_struct_cat_52_169_205_169.pkl')
cn_func = returning_pkl_file_data(r'feature_extraction/CN_func_42_79_95_79_197.pkl')
cn_struct = returning_pkl_file_data(r'feature_extraction/CN_struct_cat_42_169_205_169.pkl')

In [10]:
# for testing 
train_fmri = np.random.rand(2, 79, 95, 79, 197, 1).astype(np.float32)
val_fmri = np.random.rand(2, 79, 95, 79, 197, 1).astype(np.float32)

train_smri = np.random.rand(2, 169, 205, 169, 1).astype(np.float32)
val_smri = np.random.rand(2, 169, 205, 169, 1).astype(np.float32)

train_labels = np.random.randint(0, 2, size=(2,)).astype(np.int32)
val_labels = np.random.randint(0, 2, size=(2,)).astype(np.int32)

In [5]:
func_data = np.concat((mci_func, cn_func), axis=0, dtype=np.float16)
func_data.shape

(94, 79, 95, 79, 197)

In [6]:
struct_data = np.concat((mci_struct, cn_struct), axis=0, dtype=np.float16)
struct_data.shape

(94, 169, 205, 169)

In [7]:
all_labels = np.concat((np.zeros((len(mci_func),)), np.ones((len(cn_func),))))
all_labels.shape

(94,)

In [8]:
func_data = np.expand_dims(func_data, axis=len(func_data.shape))
struct_data = np.expand_dims(struct_data, axis=len(struct_data.shape))

func_data.shape, struct_data.shape

((94, 79, 95, 79, 197, 1), (94, 169, 205, 169, 1))

In [9]:
from sklearn.model_selection import train_test_split

func_train, func_test, struct_train, struct_test, y_train, y_test = train_test_split(
    func_data, struct_data, all_labels, test_size=0.2, random_state=42
)

func_train.shape, func_test.shape, struct_train.shape, struct_test.shape, y_train.shape, y_test.shape

((75, 79, 95, 79, 197, 1),
 (19, 79, 95, 79, 197, 1),
 (75, 169, 205, 169, 1),
 (19, 169, 205, 169, 1),
 (75,),
 (19,))

In [60]:
from tensorflow.keras.layers import ConvLSTM3D, LayerNormalization, MultiHeadAttention, Layer, Dense, Conv3D, GlobalAveragePooling3D, Concatenate, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import Input
import tensorflow as tf
import numpy as np

In [61]:
class SelfAttention3D(Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(SelfAttention3D, self).__init__()
        self.num_heads = num_heads
        self.embed_dim = embed_dim
        self.attention = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)

    def call(self, inputs):
        """
        Inputs shape: (batch_size, time_steps, depth, height, width, channels)
        Converts to 3D → Applies Self-Attention → Converts back to 5D
        """
        batch_size, time_steps, d, h, w, c = tf.unstack(tf.shape(inputs))

        # 5D -> 3D (batch_size, time_steps, features), where features = d * h * w * c
        reshaped_inputs = tf.reshape(inputs, (batch_size, time_steps, d * h * w * c))

        # Apply Self-Attention
        attended = self.attention(reshaped_inputs, reshaped_inputs)

        # 3D -> 5D (reshape back to original format)
        output = tf.reshape(attended, (batch_size, time_steps, d, h, w, c))
        return output

In [62]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, GlobalAveragePooling3D, Dense, Dropout, MultiHeadAttention, LayerNormalization, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Lambda


def build_spatiotemporal_model_fmri(input_shape):
    """
    Build a spatiotemporal model for 5D fMRI data.
    
    Args:
        input_shape (tuple): Shape of the input data, e.g., (79, 95, 79, 197, 1).
    
    Returns:
        Model: A spatiotemporal model.
    """
    inputs = Input(shape=input_shape, name="input")
    x = Lambda(lambda x: tf.transpose(x, perm=[0, 4, 1, 2, 3, 5]))(inputs)
    # 3D CNN for spatial features
    x = TimeDistributed(Conv3D(32, (3, 3, 3), activation="relu", padding="same"))(inputs)
    x = TimeDistributed(MaxPooling3D((2, 2, 2)))(x)
    x = TimeDistributed(Conv3D(64, (3, 3, 3), activation="relu", padding="same"))(x)
    x = TimeDistributed(MaxPooling3D((2, 2, 2)))(x)
    x = TimeDistributed(GlobalAveragePooling3D())(x)  # Output shape: (197, 64)
    
    # LSTM for temporal features
    x = LSTM(128, return_sequences=False)(x)  # Output shape: (128,)
    
    # Fully connected layers
    x = Dense(64, activation="relu")(x)
    x = Dense(1, activation="sigmoid")(x)
    
    return Model(inputs, x, name="Spatiotemporal_Model")

def build_3d_cnn_smri(input_shape,name):
    inputs = Input(shape=input_shape, name=f"{name}_input")
    x = Conv3D(32, (3, 3, 3), activation="relu", padding="same")(inputs)
    x = MaxPooling3D((2, 2, 2))(x)
    x = Conv3D(64, (3, 3, 3), activation="relu", padding="same")(x)
    x = MaxPooling3D((2, 2, 2))(x)
    x = GlobalAveragePooling3D()(x)
    return Model(inputs, x, name=name)

# --- Transformer for Temporal Features ---
def transformer_encoder(x,num_heads, ff_dim, dropout=0.1):
    # Multi-head attention
    # inputs = Input(shape=input_shape, name="transformer_input")
    
    return SelfAttention3D(embed_dim=64, num_heads=4)(x)

def build_transformer(input_shape, num_heads, ff_dim, num_layers):
    inputs = Input(shape=input_shape)
    x = inputs
    for _ in range(num_layers):
        x = transformer_encoder(x, num_heads, ff_dim)
    x = GlobalAveragePooling3D()(x)
    return Model(inputs, x, name="Transformer")

# --- Combined Model ---
def build_combined_model(fmri_shape, smri_shape):
    # fMRI Pathway
    # fmri_input = Input(shape=fmri_shape, name="fmri_input")
    fmri_cnn = build_spatiotemporal_model_fmri(fmri_shape)
    fmri_transformer = build_transformer(fmri_shape, num_heads=8, ff_dim=128, num_layers=2)

    # sMRI Pathway
    # smri_input = Input(shape=smri_shape, name="smri_input")
    smri_cnn = build_3d_cnn_smri(smri_shape, name="sMRI_CNN")

    # Combine Features
    combined = Concatenate()([fmri_cnn.output, fmri_transformer.output, smri_cnn.ouput])
    combined = Dense(128, activation="relu")(combined)
    combined = Dropout(0.5)(combined)
    output = Dense(1, activation="sigmoid")(combined)

    return Model(inputs=[fmri_input.input, smri_input.input], outputs=output, name="Combined_Model")


In [63]:

# Build & Compile Model
fmri_shape = (79, 95, 79, 197, 1)  # Downsampled fMRI shape
smri_shape = (169, 205, 169, 1)      # Downsampled sMRI shape
model = build_combined_model(fmri_shape, smri_shape)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

: 

In [ ]:
from skimage.transform import resize

def downsample_volume(volume, new_shape):
    return resize(volume, new_shape, mode='constant')

# Downsample fMRI data from (79, 95, 79, 197, 1) to (40, 48, 40, 100, 1)
new_shape = (40, 48, 40, 100, 1)
# func_train.shape, func_test.shape, struct_train.shape, struct_test.shape, y_train.shape, y_test.shape
func_train = np.array([downsample_volume(sample, new_shape) for sample in func_train])
func_test = np.array([downsample_volume(sample, new_shape) for sample in func_test])
func_train.shape, func_test.shape, struct_train.shape, struct_test.shape, y_train.shape, y_test.shape


NameError: name 'func_train' is not defined

In [13]:
train_fmri.shape,val_fmri.shape,train_smri.shape

((2, 79, 95, 79, 197, 1), (2, 79, 95, 79, 197, 1), (2, 169, 205, 169, 1))

In [ ]:
# --- fMRI Model (ConvLSTM) ---
def build_fmri_model():
    fmri_input = Input(shape=(79, 95, 79, 197, 1), name="fmri_input", dtype=tf.float16)
    x = Lambda(lambda x: tf.transpose(x, perm=[0, 4, 1, 2, 3, 5]))(fmri_input)
    # Apply Conv3D independently to each time step
    x = TimeDistributed(Conv3D(32, (3,3,3), padding="same", activation="relu"))(x)
    x = TimeDistributed(MaxPooling3D(2))(x)  # Downsample spatial dimensions
    x = TimeDistributed(Conv3D(64, (3,3,3), padding="same", activation="relu"))(x)
    x = TimeDistributed(GlobalAveragePooling3D())(x)  # Shape: (batch, time=197, 64)
    x = TimeDistributed(GlobalAveragePooling3D())(x)  # Shape: (batch, time=197, 64)

    # Temporal modeling with LSTM
    # x = LSTM(128)(x)  # Output shape: (batch, 128)
    x = Flatten()(x)
    x = Dense(128)(x)
    return Model(inputs=fmri_input, outputs=x, name="fMRI_Model")

# --- sMRI Model (3D CNN) ---
def build_smri_model():
    smri_input = Input(shape=(169, 205, 169, 1), name="smri_input", dtype=tf.float16)

    y = Conv3D(filters=32, kernel_size=(3,3,3), activation="relu", padding="valid")(smri_input)
    y = BatchNormalization()(y)
    y = Conv3D(filters=32, kernel_size=(3,3,3), activation="relu", padding="valid")(y)
    y = BatchNormalization()(y)

    y = GlobalAveragePooling3D()(y)
    y = Dense(128, activation="relu")(y)

    return Model(inputs=smri_input, outputs=y, name="sMRI_Model")

# --- Combine fMRI & sMRI Models ---
def build_combined_model():
    fmri_model = build_fmri_model()
    smri_model = build_smri_model()

    combined = Concatenate()([fmri_model.output, smri_model.output])
    combined = Dense(128, activation="relu")(combined)
    combined = Dropout(0.5)(combined)
    output = Dense(1, activation="sigmoid")(combined)

    model = Model(inputs=[fmri_model.input, smri_model.input], outputs=output, name="Combined_Model")
    return model

# Build & Compile Model
model = build_combined_model()
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()


Model: "Combined_Model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ fmri_input          │ (None, 79, 95,    │          0 │ -                 │
│ (InputLayer)        │ 79, 197, 1)       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cast (Cast)         │ (None, 79, 95,    │          0 │ fmri_input[0][0]  │
│                     │ 79, 197, 1)       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ smri_input          │ (None, 169, 205,  │          0 │ -                 │
│ (InputLayer)        │ 169, 1)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 197, 79,   │          0 │ cast[0][0]        │
│                     │ 95, 79, 1)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cast_1 (Cast)       │ (None, 169, 205,  │          0 │ smri_input[0][0]  │
│                     │ 169, 1)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, 197, 79,   │        896 │ lambda[0][0]      │
│ (TimeDistributed)   │ 95, 79, 32)       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_2 (Conv3D)   │ (None, 167, 203,  │        896 │ cast_1[0][0]      │
│                     │ 167, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_1  │ (None, 197, 39,   │          0 │ time_distributed… │
│ (TimeDistributed)   │ 47, 39, 32)       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 167, 203,  │        128 │ conv3d_2[0][0]    │
│ (BatchNormalizatio… │ 167, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_2  │ (None, 197, 39,   │     55,360 │ time_distributed… │
│ (TimeDistributed)   │ 47, 39, 64)       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_3 (Conv3D)   │ (None, 165, 201,  │     27,680 │ batch_normalizat… │
│                     │ 165, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_3  │ (None, 197, 64)   │          0 │ time_distributed… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 165, 201,  │        128 │ conv3d_3[0][0]    │
│ (BatchNormalizatio… │ 165, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 12608)     │          0 │ time_distributed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 32)        │          0 │ batch_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │  1,613,952 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │      4,224 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 256)       │          0 │ dense[0][0],    

 Total params: 1,736,289 (6.62 MB)

 Trainable params: 1,736,161 (6.62 MB)

 Non-trainable params: 128 (512.00 B)

In [ ]:
fmri_data = np.random.rand(100, 5, 4, 4, 4, 1)
smri_data = np.random.rand(100, 10, 10, 10, 1)
labels = np.random.randint(0, 2, size=(100,))
fmri_data.shape, smri_data.shape, labels.shape

((100, 5, 4, 4, 4, 1), (100, 10, 10, 10, 1), (100,))

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_fmri, val_fmri, train_smri, val_smri, train_labels, val_labels = train_test_split(fmri_data, smri_data, labels, test_size=0.2, random_state=42)
train_fmri.shape, val_fmri.shape, train_smri.shape, val_smri.shape, train_labels.shape, val_labels.shape

((80, 5, 4, 4, 4, 1),
 (20, 5, 4, 4, 4, 1),
 (80, 10, 10, 10, 1),
 (20, 10, 10, 10, 1),
 (80,),
 (20,))

In [12]:
history = model.fit(
    {"fmri_input": func_train, "smri_input": struct_train},  # Dictionary format for inputs
    y_train,  # Output labels
    batch_size=8,
    epochs=10,
    validation_data=(
        {"fmri_input": func_test, "smri_input": struct_test},
        y_test
    ),
    verbose=1
)

Epoch 1/10


: 

In [4]:
# Second appraoch

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Flatten, Dropout, Conv3D, MaxPooling3D, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Define input shapes
func_input_shape = (79, 95, 79, 197, 1)
struct_input_shape = (169, 205, 169, 1)

def create_model():
    # Functional data branch
    func_input = Input(shape=func_input_shape)
    
    # Use 3D convolutions to reduce dimensionality first
    x_func = Conv3D(16, kernel_size=(3, 3, 3), activation='relu')(func_input)
    x_func = MaxPooling3D(pool_size=(2, 2, 2))(x_func)
    x_func = Conv3D(32, kernel_size=(3, 3, 3), activation='relu')(x_func)
    x_func = MaxPooling3D(pool_size=(2, 2, 2))(x_func)
    
    # Reshape for LSTM (sequence, features)
    x_func = Flatten()(x_func)
    x_func = Dense(512, activation='relu')(x_func)
    x_func = Dropout(0.3)(x_func)
    x_func = tf.keras.layers.Reshape((-1, 512))(x_func)  # Reshape for LSTM
    x_func = LSTM(128, return_sequences=False)(x_func)
    
    # Structural data branch
    struct_input = Input(shape=struct_input_shape)
    
    # Use 3D convolutions to reduce dimensionality
    x_struct = Conv3D(16, kernel_size=(3, 3, 3), activation='relu')(struct_input)
    x_struct = MaxPooling3D(pool_size=(2, 2, 2))(x_struct)
    x_struct = Conv3D(32, kernel_size=(3, 3, 3), activation='relu')(x_struct)
    x_struct = MaxPooling3D(pool_size=(2, 2, 2))(x_struct)
    
    # Reshape for LSTM (sequence, features)
    x_struct = Flatten()(x_struct)
    x_struct = Dense(512, activation='relu')(x_struct)
    x_struct = Dropout(0.3)(x_struct)
    x_struct = tf.keras.layers.Reshape((-1, 512))(x_struct)  # Reshape for LSTM
    x_struct = LSTM(128, return_sequences=False)(x_struct)
    
    # Merge branches
    merged = Concatenate()([x_func, x_struct])
    
    # Final classification layers
    x = Dense(64, activation='relu')(merged)
    x = Dropout(0.5)(x)
    output = Dense(1, activation='sigmoid')(x)
    
    # Create model
    model = Model(inputs=[func_input, struct_input], outputs=output)
    
    # Use mixed precision for faster training
    tf.keras.mixed_precision.set_global_policy('mixed_float16')
    
    # Compile with optimization settings
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC()]
    )
    
    return model

# Train the model with optimizations
def train_model(model, func_train, struct_train, y_train, func_test, struct_test, y_test, epochs=50, batch_size=8):
    # Callbacks for optimization
    callbacks = [
        EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-6,
            verbose=1
        )
    ]
    
    # Use TensorFlow data API for faster data loading
    train_dataset = tf.data.Dataset.from_tensor_slices(
        ((func_train, struct_train), y_train)
    ).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    test_dataset = tf.data.Dataset.from_tensor_slices(
        ((func_test, struct_test), y_test)
    ).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    # Train the model
    history = model.fit(
        train_dataset,
        epochs=epochs,
        validation_data=test_dataset,
        callbacks=callbacks,
        verbose=1
    )
    
    return model, history
    

In [ ]:
model2 = create_model()
model2.summary()

In [ ]:
history2 = model2.fit(
    {"fmri_input": func_train, "smri_input": struct_train},  # Dictionary format for inputs
    y_train,  # Output labels
    batch_size=8,
    epochs=10,
    validation_data=(
        {"fmri_input": func_test, "smri_input": struct_test},
        y_test
    ),
    verbose=1
)